In [347]:
import math
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
#from scipy import io
#from collections import Counter
from matplotlib import pyplot as plt
from signet.cluster import Cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [348]:
df_small = pd.read_csv("New_ Microsoft_ Excel_ Worksheet.csv")
#df_small = pd.read_csv("test11.csv")
df_small = df_small.drop_duplicates(inplace = False)
df_small_p = df_small[df_small['sign'] == 1]
df_small_n = df_small[df_small['sign'] == -1]

In [349]:
df_small_train_p, df_small_test_p, df_small_train_label_p, df_small_test_label_p = train_test_split(df_small_p, df_small_p['source'], test_size = 0.40)
df_small_train_n, df_small_test_n, df_small_train_label_n, df_small_test_label_n = train_test_split(df_small_n, df_small_n['source'], test_size = 0.40)
df_small_train_final = pd.concat([df_small_train_p, df_small_train_n])
df_small_test_final = pd.concat([df_small_test_p, df_small_test_n])

In [350]:
#df_small_test_final.head()

In [351]:
def relabel(df):
    source_old_labels = df["source"].drop_duplicates().to_numpy()
    #print(source_old_labels)
    target_old_labels = df["target"].drop_duplicates().to_numpy()
    all_old_labels = np.union1d(source_old_labels, target_old_labels)
    index_map = dict(enumerate(all_old_labels))
    new_index_map = dict([(value, key) for key, value in index_map.items()])
    #print(all_old_labels)
    new_df = df.copy()
    for index, row in new_df.iterrows():
        #print(row["source"])
        row["source"] = new_index_map[row["source"]]
        row["target"] = new_index_map[row["target"]]
    return new_index_map, new_df

In [352]:
ind_map, df_small_test_final = relabel(df_small_test_final)

In [353]:
G = nx.DiGraph()
G = nx.from_pandas_edgelist(df = df_small_test_final, source = 'source', target = 'target', edge_attr = 'sign', create_using = nx.DiGraph(), edge_key = 'sign')

In [354]:
nodes_no = nx.number_of_nodes(G)
edges_no = nx.number_of_edges(G)

In [355]:
print('number of nodes G =',nx.number_of_nodes(G) )
print('number of edges G =',nx.number_of_edges(G) )
print('Density of G:', nx.density(G))

number of nodes G = 693
number of edges G = 807
Density of G: 0.0016828065961013937


In [356]:
rows = df_small_test_final.loc[:, "source"].to_numpy()
cols = df_small_test_final.loc[:, "target"].to_numpy()
sign = df_small_test_final.loc[:, "sign"].to_numpy()

In [357]:
p_rows = [rows[i] for i in range(len(rows)) if sign[i] == 1]
p_cols = [cols[i] for i in range(len(cols)) if sign[i] == 1]
p_data = np.ones(len(p_rows))

n_rows = [rows[i] for i in range(len(rows)) if sign[i] == -1]
n_cols = [cols[i] for i in range(len(cols)) if sign[i] == -1]
n_data = np.ones(len(n_rows))

In [358]:
A_p = sc.sparse.csc_matrix((p_data, (p_rows, p_cols)), shape = (nodes_no , nodes_no))
A_n = sc.sparse.csc_matrix((n_data, (n_rows, n_cols)), shape = (nodes_no , nodes_no))
#A_p = A_p+A_p.T
#A_n = A_n+A_n.T
#print("A_p",A_p)
#print("A_n",A_n)

In [359]:
print(p_data.shape)
print(n_data.shape)

(696,)
(111,)


In [360]:
def getNeighborsOfANode(G, u):
    x = []
    for v in G.nodes():
        if ((v, u) in G.edges()):
            x.append(v)
    return np.array(x)






def NodeswithoutInAndOutneighbors(G, sign_tag = 'sign'):
    nodes_no = nx.number_of_nodes(G)
    sign_map = nx.get_edge_attributes(G, sign_tag)
    n = 0
    for u in G.nodes():
        for v in G.nodes():
            #print("u: ", u, ", v: ", v)
            if (u != v):
                u_neighbors = set(G.adj[u])
                v_neighbors = set(G.adj[v])
                uv_neighbors = list(u_neighbors.intersection(v_neighbors))
                u_inneighbors = set(getNeighborsOfANode(G, u))
                v_inneighbors = set(getNeighborsOfANode(G, v))
                uv_inneighbors = list(u_inneighbors.intersection(v_inneighbors))
                if (len(uv_neighbors) + len(uv_inneighbors)) == 0 and (u,v) in G.edges():
                    n = n+1
                    print("(u,v):" , (u,v))
    return n

In [361]:
#print(NodeswithoutInAndOutneighbors(G, sign_tag = 'sign'))

In [362]:
c = Cluster((A_p, A_n))

In [363]:
#for i in range(1,20):
    #print(c.spectral_cluster_bnc(k = 2, normalisation = 'sym', eigens = None, mi = None))

In [364]:
def getClusters(G, k):
    spec_clus = c.spectral_cluster_adjacency(k = 5, normalisation = 'sym_sep', eigens = None, mi = None)
    clusters = []
    for j in range(k):
        clusters.append([i for i in G.nodes() if spec_clus[int(i - 1)] == j])
    return np.array(clusters, dtype = np.ndarray)

In [365]:
clusters = getClusters(G, 5)
cluster1 = clusters[0]
cluster2 = clusters[1]
cluster3 = clusters[2]
cluster4 = clusters[3]
cluster5 = clusters[4]

#print("cluster1:",cluster1)
#print()
#print("cluster2:",cluster2)
#print()
#print("cluster3:",cluster3)
#print()
#print("cluster4:",cluster4)
#print()
#print("cluster5:",cluster5)


In [366]:
print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
print(len(cluster4))
print(len(cluster5))

40
281
326
2
44


In [367]:
def getCommonNeighbors(G, cl1, cl2):
    x = set([])
    for i in cl1:
        x = x.union(set(G.adj[i]))
    #print('x: ', x)
    y = set([])
    for j in cl2:
        y = y.union(set(G.adj[j]))
    #print('y: ', y)
    return list(x.intersection(y))

def locationOfANode(clusters, u):
    n = clusters.shape[0]
    x = np.zeros(n)
    r = -1
    for i in range(n):
        if u in clusters[i]:
            r = i
    return r

def getClusterSimiliarity(Cluster, common_neighbors, sign_tag = 'sign'):
    sign_map = nx.get_edge_attributes(G, sign_tag)
    epsilon = 10 ** (-5)
    y = []
    for v in common_neighbors:
        # adding an epsilon to prevent mean of an empty array
        x = [epsilon]
        #x = []
        for u in Cluster:
            if (u, v) in G.edges():
                x.append(sign_map[(u, v)])
                #print("u", u , "v" , v)
        y.append(np.mean(np.array(x)))
    return np.array(y)


def getSimiliarity(y1, y2):
    alpha = np.dot(y1.T, y2)
    beta = np.dot(y1.T, y1)
    gamma = np.dot(y2.T, y2)
    if (beta * gamma < 0):
        print('it"s negative: ', beta * gamma)
    epsilon = 10 ** (-5)
    return alpha / (np.sqrt((beta * gamma)) + epsilon)


# main function to calculate the similiarity between two clusters
def getInterClusterSimiliarity(cl1, cl2, common_neighbors):
    y1 = getClusterSimiliarity(cl1, common_neighbors)
    y2 = getClusterSimiliarity(cl2, common_neighbors)
    return getSimiliarity(y1, y2)

# calculates similiarities between each two clusters and returns a matrix
def getAllSimiliaritiesBetweenClusters(G, clusters):
    clusters_no = clusters.shape[0]
    similiarities = np.zeros(shape = (clusters_no, clusters_no))
    for i in range(clusters_no):
        for j in range(i, clusters_no):
            common_neighbors = getCommonNeighbors(G, clusters[i], clusters[j])
            current_similiarity = getInterClusterSimiliarity(clusters[i], clusters[j], common_neighbors)
           # similiarities[i][j] = similiarities[j][i] = current_similiarity
            if (math.isnan(current_similiarity)):
                similiarities[i][j] = similiarities[j][i] = 0
            else:
                similiarities[i][j] = similiarities[j][i] = current_similiarity
    return similiarities

In [368]:
common_neighbors = getCommonNeighbors(G, cluster1, cluster2)
common_neighbors
y1 = getClusterSimiliarity(cluster1, common_neighbors)
y2 = getClusterSimiliarity(cluster2, common_neighbors)
similarity = getInterClusterSimiliarity(cluster1, cluster2, common_neighbors)
similarity 
matrix_of_similarity = getAllSimiliaritiesBetweenClusters(G, clusters)
matrix_of_similarity

array([[ 1.  ,  0.58,  0.43,  0.  , -0.38],
       [ 0.58,  1.  ,  0.67,  0.  ,  0.27],
       [ 0.43,  0.67,  1.  ,  0.  ,  0.51],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [-0.38,  0.27,  0.51,  0.  ,  1.  ]])

In [369]:
def getNeighborsOfANode(G, u):
    x = []
    for v in G.nodes():
        if ((v, u) in G.edges()):
            x.append(v)
    return np.array(x)

In [370]:

def locationOfArrayOfNode(clusters, U):
    x = np.zeros(len(U))
    for i in range(len(x)):
        x[i] = locationOfANode(clusters, U[i])
    return x

In [371]:
def signPrediction(G, k, u, v, threshold, w1, sign_tag = "sign"):
    sign_map = nx.get_edge_attributes(G, sign_tag)
    main_clusters = getClusters(G, k)
    u_c = locationOfANode(main_clusters, u)
    u_v = locationOfANode(main_clusters, v)
    cl1 = main_clusters[u_c]
    cl2 = main_clusters[u_v]
    s_AB = getCommonNeighbors(G, cl1, cl2)
    #print(s_AB)
    S = getNeighborsOfANode(G, v)
    S_c = locationOfArrayOfNode(main_clusters, S)
    n = len(S)
    x = np.zeros(n)
    for i in range(n):
        x[i] = getInterClusterSimiliarity(main_clusters[u_c], main_clusters[int(S_c[i])], s_AB)
        if locationOfANode(main_clusters, u) == locationOfANode(main_clusters, S_c[i]):
            x[i] = w1 * x[i] 
        else:
            x[i] = (1-w1) * x[i]
        
    r = np.zeros(n)
    for i in range(n):
        r[i] = sign_map[(S[i], v)]
    #print('x: ', x)
    #print('r: ', r)
    #if (np.sum(x) == 0):
    #    print('it is zero')
    epsilon = 10 ** (-5)
    result = (np.dot(x.T, r)) / (np.sum(x) + epsilon)
    if result > threshold:
        return 1
    else:
        return -1

In [372]:
#zz = np.zeros(df_small_test_final.shape[0])
#df_small_test_final['sign prediction'] = df_small_test_final.apply(lambda row : signPrediction(G, 5, row['source'], row['target'], 0, 0.75), axis = 1)
#confusion_matrix(df_small_test_final['sign'], df_small_test_final['sign prediction'])
#acc_temp = accuracy_score(df_small_test_final['sign'], df_small_test_final['sign prediction'],  normalize=True, sample_weight=None)
#f_temp = f1_score(df_small_test_final['sign'], df_small_test_final['sign prediction'])

In [373]:
#print(acc_temp)
#print(f_temp)

0.9454770755885997
0.9685714285714285


In [374]:
acc_x = [i / 10 for i in range(0, 11)]
f_x = [i / 10 for i in range(0, 11)]
acc_y = []
f_y = []
for w in [i / 10 for i in range(0, 11)]:
    zz = np.zeros(df_small_test_final.shape[0])
    df_small_test_final['sign prediction'] = df_small_test_final.apply(lambda row : signPrediction(G, 5, row['source'], row['target'], 0, w), axis = 1)
    confusion_matrix(df_small_test_final['sign'], df_small_test_final['sign prediction'])
    acc_temp = accuracy_score(df_small_test_final['sign'], df_small_test_final['sign prediction'],  normalize=True, sample_weight=None)
    f_temp = f1_score(df_small_test_final['sign'], df_small_test_final['sign prediction'])
    acc_y.append(acc_temp)
    f_y.append(f_temp)
    

KeyboardInterrupt: 

In [ ]:
plt.plot(acc_x, acc_y)
plt.title('Accuracy per weight')

In [ ]:
plt.plot(f_x, f_y)
plt.title('F1 per weight')

In [ ]:
print(acc_y)

In [ ]:
print(f_y)